# Brain Tumor Detection- Keras/Pytorch 🧠🧠
* https://www.kaggle.com/code/fahadmehfoooz/brain-tumor-detection-keras-pytorch
---

### data
* 
* 

## 1. Importing Modules

In [1]:
import os, glob, pathlib # 파일 위치 찾기, 입출력 등
from tqdm import tqdm # 진행 상황 표시 모듈 -> 반복문에서 현재 상태를 시각화, 진행률과 소요 시간 등 표시
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

In [2]:
import cv2
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torch.autograd import Variable

import torchvision
from torchvision.transforms import transforms

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D
from tensorflow.keras.optimizers import Adam
from kerastuner import RandomSearch

## 2. Data

### 1) Loading

In [9]:
# kaggle api
!kaggle datasets download sartajbhuvaji/brain-tumor-classification-mri

Dataset URL: https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri
License(s): MIT




  0%|          | 0.00/86.8M [00:00<?, ?B/s]
  1%|          | 1.00M/86.8M [00:00<00:28, 3.19MB/s]
  2%|▏         | 2.00M/86.8M [00:00<00:17, 4.99MB/s]
  5%|▍         | 4.00M/86.8M [00:00<00:11, 7.33MB/s]
  6%|▌         | 5.00M/86.8M [00:00<00:10, 8.03MB/s]
  7%|▋         | 6.00M/86.8M [00:00<00:09, 8.54MB/s]
  8%|▊         | 7.00M/86.8M [00:00<00:09, 8.89MB/s]
  9%|▉         | 8.00M/86.8M [00:01<00:09, 9.17MB/s]
 10%|█         | 9.00M/86.8M [00:01<00:08, 9.41MB/s]
 12%|█▏        | 10.0M/86.8M [00:01<00:08, 9.50MB/s]
 13%|█▎        | 11.0M/86.8M [00:01<00:08, 9.66MB/s]
 14%|█▍        | 12.0M/86.8M [00:01<00:08, 9.73MB/s]
 15%|█▍        | 13.0M/86.8M [00:01<00:07, 9.75MB/s]
 16%|█▌        | 14.0M/86.8M [00:01<00:07, 9.79MB/s]
 17%|█▋        | 15.0M/86.8M [00:01<00:07, 9.82MB/s]
 18%|█▊        | 16.0M/86.8M [00:01<00:07, 9.68MB/s]
 21%|██        | 18.0M/86.8M [00:02<00:07, 9.89MB/s]
 22%|██▏       | 19.0M/86.8M [00:02<00:07, 9.93MB/s]
 23%|██▎       | 20.0M/86.8M [00:02<00:07, 9.90MB/s]
 

In [10]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("sartajbhuvaji/brain-tumor-classification-mri")

# print("Path to dataset files:", path)

100%|██████████| 86.8M/86.8M [00:09<00:00, 9.55MB/s]

Extracting model files...


Path to dataset files: C:\Users\wjddl\.cache\kagglehub\datasets\sartajbhuvaji\brain-tumor-classification-mri\versions\2


In [11]:
PATH_TRAIN = '/data/brain-tumor-classification-mri/Training'
PATH_TEST = '/data/brain-tumor-classification-mri/Testing'

In [12]:
train_img = []
train_labels = []

test_img = []
test_labels = []

img_size= 300

for i in os.listdir(PATH_TRAIN):
    for j in os.listdir(PATH_TRAIN+i):
        train_img.append (cv2.resize(cv2.imread(PATH_TRAIN+i+'/'+j), (img_size,img_size))) 
        train_labels.append(i)
        
for i in os.listdir(PATH_TEST):
    for j in os.listdir(PATH_TEST+i):
        test_img.append (cv2.resize(cv2.imread(PATH_TEST+i+'/'+j), (img_size,img_size))) 
        test_labels.append(i)
        
train_img = (np.array(train_img))
test_img = (np.array(test_img))


train_labels_encoded = [0 if category == 'no_tumor' else(1 if category == 'glioma_tumor' else(2 if category=='meningioma_tumor' else 3)) for category in list(train_labels)]
test_labels_encoded = [0 if category == 'no_tumor' else(1 if category == 'glioma_tumor' else(2 if category=='meningioma_tumor' else 3)) for category in list(test_labels)]

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '/data/brain-tumor-classification-mri/Training'

## 3-1. Tensorflow(Keras)